<a href="https://colab.research.google.com/github/SwethaMagesh/Dots_And_Boxes_RL/blob/main/Version2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitstring

## ML ASSIGNMENT BATCH 8
- Deepthishree GS
- Harshini S
- Iswarya GP
- Janani R
- Swetha M

### ***TOPIC*** - REINFORCEMENT LEARNING
### ***Problem Taken*** : Playing Dots and Boxes using a Q learning Agent


## Importing Libraries 

In [2]:
from bitstring import BitArray
import numpy as np
import random
import json

## Global Variables declaration


In [3]:
'''
3x3 dots and boxes 
4 boxes 
12 lines to choose from for each move
2 power 12 game states for the agent to learn from
DRAW / TIE is available in this version of the game
Reward 100 and Penalty -100
'''
QTable=dict()
VISIT_TABLE=dict()
DISCOUNT_RATE=0.8
LEARNING_RATE=0.2

### Main Function for LEARNER
- Algorithm is play games repeatedly and update the Q Table 
- Calls subfunction playGame

In [17]:
def learner(gameCount=1000):
    count=1
    choiceOfMove=['random','simple','qlearner']
    while count<gameCount:
        print("GAME ",count)
        if count<1000:
            playGame(choiceOfMove[0],False)
        elif count<5000:
            playGame(choiceOfMove[1],False)
        else:
            playGame(choiceOfMove[2],False)
        count+=1
        with open('Qtable.json','w') as jsonfile:
          json.dump(QTable,jsonfile)



    
        
        

### Initialises the Main Q Table
- Q TABLE usually has states as rows and actions as columns and the entries are the corresponding Q values

- Since in a game, only legal moves are the actions, the number of columns(moves) for each state is different

- So, a hash table is constructed with keys as states and values as action:qvalue pairs


In [5]:

def initialiseQTable():
    global QTable
    global VISIT_TABLE
    for i in range(4096):
        currentState=BitArray(uint=i, length=12)
        actionQvalues=dict()
        # find possible actions
        bitString=currentState.bin
        for index in range(len(bitString)):
            if bitString[index]=='0':
                actionQvalues[index+1]=0
        QTable[currentState.bin]=actionQvalues
        VISIT_TABLE[currentState.bin]=0
        QTable['111111111111']={0:0}
initialiseQTable()

                
        
    

### Function used by QLearner to select the action or move which has maximum QValue

In [16]:
def bestActionForState(state):
    maxQval=-200 #negative inf
    bestAction=0
    for action,qval in QTable[state].items():
        if qval>maxQval:
            maxQval=qval
            bestAction=action
    return bestAction



    
    

### UPDATE QTABLE
- This function uses the formula for QLearning and updates the moves in reverse chronological order
- This is called after each game is over
- This finetuning is what makes the agent learn the game

In [ ]:

def updateQTable(memoryOfPlayerMoves):
    global QTable
    memoryOfPlayerMoves=memoryOfPlayerMoves[::-1] #reverse chronological order
    for state,action,nextState,reward in memoryOfPlayerMoves:
        maxQValForNextState=QTable[nextState][bestActionForState(nextState)]
        existingValue=(1-LEARNING_RATE)*QTable[state][action]
        updateValue=LEARNING_RATE*(reward+DISCOUNT_RATE*maxQValForNextState)
        QTable[state][action]=existingValue+updateValue
        VISIT_TABLE[state]+=1

### Print the board state so that the human can understand what are the boxes formed and what could be the potential next move during a game

In [7]:
def printTable(currentState):
    count=1
    m=7
    for i in range(3):
        for j in range(3):
            print(".\t", end=" ")
            if j<2:
                if currentState[count-1]=='0':
                    print(str(count)+"\t",end=" ") 
                else:
                    print("--\t",end=" ")
                count+=1
                
        print("\n")
        if i!=2:
            for k in range(3):
                if currentState[m-1]=='0':
                    print(str(m)+"\t\t",end=" ")
                else:
                    print("|\t\t",end=" ")
                m+=1
        print("\n")
# printTable('111111111111')

### MakeMove or ChooseAction
- First few games, learner uses random moves
- Next it uses an approach to select states with very less visit count. 
- These are for exploration
- After that, the game is carried forward by the QLearner

- Human can choose any of the three for choice of making move for his/her opponent

In [8]:

def MakeMove(currentState,choice):
    # possible actions
    possibleActions=list(QTable[currentState].keys())
    # print(possibleActions)
    #choiceOfMove=['random','simple','qlearner']
    if choice=='random':
        return possibleActions[random.randint(0,len(possibleActions)-1)]
        
    if choice=='simple':
        minCount=1000000
        action=0
        for action in possibleActions:
            if VISIT_TABLE[transition(currentState,action)]<minCount:
                minCount=VISIT_TABLE[transition(currentState,action)]
                bestAction=action     
        return bestAction
    if choice=='qlearner':
        return bestActionForState(currentState)  

    
 

### Checks if the game is over

In [9]:
def isBoardFinished(boardState):
    return boardState.bin=='111111111111'

### Transition function
- Takes input as current state and move and gives the next state as output

In [10]:
def transition(currentState,currentMove):
    temp=currentState
    temp=temp[:currentMove-1]+'1'+temp[currentMove:]
#     print("Transition from state",currentState,"Is ",temp)
    return temp

#print(transition(BitArray(uint=10,length=12),11))

### Counts the number of boxes formed during the move to update points

In [11]:
def numberOfNewBoxFormed(box,newState):
    count=0
    boxBounds=[[1,3,7,8],[2,4,8,9],[3,5,10,11],[4,6,11,12]]
    for i in range(len(boxBounds)):
      if box[i]==0:
        flag=True
        for lineNumber in boxBounds[i]:
          if newState[lineNumber-1]!='1':
            flag=False
            break
        if flag==True:
          count+=1
          box[i]=1
    return count
        
                

### Main function where the game is played

Algo
- init board state
- init players as p1,p2 or p1,human
- init current player = p1
- init box [0 0 0 0]
- while not final state: 
    - if cp != 'human' : 
        currentplayer.make move // simple, q learner , random .. 3 boxes..
    - else:
        accept input
    - update currentplayer.memory
    - Check if new box formed, update player score
    - else 
    -   toggle the currentplayer
    - update board state with current move
- update QTABLE with rewards and penalties


In [12]:
def playGame(choiceOfMove,withHuman=False):
    #Game init
    player=["p1","p2"]
    if withHuman:
        player[1]='human'
    currentPlayer="p1"
    box=[0,0,0,0]
    currentState=BitArray(uint=0, length=12)
    memory={player[0]:[],player[1]:[]}
    score={player[0]:0,player[1]:0}
    while isBoardFinished(currentState)==False:
        # print("Playing game")
        if(currentPlayer!="human"):
            currentMove=MakeMove(currentState.bin,choiceOfMove)
        else:
          possibleActions=list(QTable[currentState.bin].keys())
          currentMove=int(input("Enter the line number:"))
          while currentMove not in possibleActions:
            currentMove=int(input("INVALID!Enter the line number:"))

        newState=transition(currentState.bin,currentMove) 
        memory[currentPlayer].append([currentState.bin,currentMove,newState,0])
        numberOfBoxes=numberOfNewBoxFormed(box,newState)
        if numberOfBoxes>0:#Update this......:)
            score[currentPlayer]+=numberOfBoxes*2
            memory[currentPlayer][:-1][0][-1]=100
            memory[player[1-player.index(currentPlayer)]][:-1][0][-1]=-100

        else:
            currentPlayer=player[1-player.index(currentPlayer)]

        currentState=BitArray(uint=int(newState,2),length=12)
        if player[1]=='human':
            
            printTable(currentState.bin)
            print("Score of ",player[0]," is ",score[player[0]])
            print("Score of ",player[1]," is ",score[player[1]])
            if isBoardFinished(currentState)==False:
              print("Next turn for ",currentPlayer)

    #find winner 
    result=""
    if score[player[0]]==score[player[1]]:
        result='TIE'
        print(result)
    else:
        result=max(score,key=score.get)
        print("Winner is ",result)
    #print(memory[player[0]])
    if result=='TIE':
        memory[player[0]][:-1][0][-1]=50
        memory[player[1]][:-1][0][-1]=50
    else: 
        memory[result][:-1][0][-1]=100
        memory[player[1-player.index(result)]][:-1][0][-1]=-100    
    #put reward and penalties in last row of the memory table
    updateQTable(memory[player[0]])
    updateQTable(memory[player[1]])





### Learn and test performance

In [ ]:
# initialiseQTable()
learner(gameCount=10000)

playGame(choiceOfMove='qlearner',withHuman=True)

### Save the QTABLE and print visit table


In [19]:
# playGame(choiceOfMove='qlearner',withHuman=True)

with open('Qtable.json','w') as jsonfile:
  json.dump(QTable,jsonfile)

print(VISIT_TABLE)

{'000000000000': 12030, '000000000001': 585, '000000000010': 588, '000000000011': 106, '000000000100': 585, '000000000101': 106, '000000000110': 106, '000000000111': 31, '000000001000': 586, '000000001001': 106, '000000001010': 107, '000000001011': 31, '000000001100': 106, '000000001101': 31, '000000001110': 32, '000000001111': 14, '000000010000': 586, '000000010001': 106, '000000010010': 106, '000000010011': 31, '000000010100': 106, '000000010101': 31, '000000010110': 31, '000000010111': 13, '000000011000': 106, '000000011001': 31, '000000011010': 31, '000000011011': 13, '000000011100': 32, '000000011101': 14, '000000011110': 14, '000000011111': 8, '000000100000': 5578, '000000100001': 106, '000000100010': 107, '000000100011': 31, '000000100100': 106, '000000100101': 31, '000000100110': 31, '000000100111': 13, '000000101000': 5097, '000000101001': 31, '000000101010': 31, '000000101011': 13, '000000101100': 32, '000000101101': 14, '000000101110': 14, '000000101111': 8, '000000110000': 